In [1]:
# Carregando a base de dados
import pandas as pd
import sqlite3

con = sqlite3.connect("../database.sqlite3")
df = pd.read_sql_query("SELECT * from vendas", con)

con.close()

In [2]:
# Utilizado para identificar produtos com ciclos completos
df_grupo = pd.read_csv('../df_grupo.csv',index_col='Unnamed: 0')

In [3]:
# Eliminando variáveis estáticas
df = df.drop(['DES_CATEGORIA_MATERIAL','DES_MARCA_MATERIAL','FLG_CAMPANHA_MKT_E','FLG_CAMPANHA_MKT_F'],axis=1)

In [4]:
# Definindo base de treino e base de teste
df_train = df[(df['COD_MATERIAL'].isin(df_grupo.index)) & (~df['COD_CICLO'].isin([202016,202017,202101]))].sort_values(by='COD_CICLO')
df_test = df[(df['COD_MATERIAL'].isin(df_grupo.index)) & (df['COD_CICLO'].isin([202016,202017,202101]))].sort_values(by='COD_CICLO')

In [5]:
# Função necessária para processamento das variáveis indicadores.
def preprocessamento_indicadores(x):
    if x>= 0.5:
        return 1
    else:
        return 0

In [6]:
# Bibliotecas para processo de modelagem
from sklearn.tree import DecisionTreeRegressor
from tqdm import tqdm
from sklearn.model_selection import TimeSeriesSplit
import numpy as np
from sklearn.metrics import mean_squared_error

In [7]:
# Feature Engineering, Cross-Validation

produto = {}

for i in tqdm(df_grupo.Grupo.index):
    df_aux = df_train[df_train['COD_MATERIAL'] == i]
    df_aux = df_aux.reset_index(drop=True)
    df_aux['ANO'] = df_aux['COD_CICLO'].apply(lambda x: int(str(x)[:4]))
    df_aux['CICLO'] = df_aux['COD_CICLO'].apply(lambda x: int(str(x)[4:]))
    df_aux['FLG_CAMPANHA_MKT_A'] = df_aux['FLG_CAMPANHA_MKT_A'].apply(lambda x: preprocessamento_indicadores(x))
    df_aux['FLG_CAMPANHA_MKT_B'] = df_aux['FLG_CAMPANHA_MKT_B'].apply(lambda x: preprocessamento_indicadores(x))
    df_aux['FLG_CAMPANHA_MKT_C'] = df_aux['FLG_CAMPANHA_MKT_C'].apply(lambda x: preprocessamento_indicadores(x))
    df_aux['FLG_CAMPANHA_MKT_D'] = df_aux['FLG_CAMPANHA_MKT_D'].apply(lambda x: preprocessamento_indicadores(x))
    df_aux['PCT_DESCONTO'].fillna(0,inplace=True)
    df_aux = df_aux.drop(['COD_CICLO','COD_MATERIAL'],axis=1)
    
    
    X = df_aux.drop('QT_VENDA',axis=1)
    y = df_aux['QT_VENDA']

    # Definindo o método de Cross-Validation que será utilizado na otimização dos parâmetros
    cv = TimeSeriesSplit(n_splits=3)
    # RMSE em cada fold
    cv_scores = np.empty(3)
    
    for idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        model = DecisionTreeRegressor()
        model.fit(X_train,y_train)

        preds = model.predict(X_test)
        cv_scores[idx] = mean_squared_error(y_test, preds,squared=False)
    
    produto[i] = [np.mean(cv_scores),model.fit(X,y)]

100%|████████████████████████████████████████████████████████████████████████████████| 329/329 [00:10<00:00, 31.00it/s]


In [8]:
# Produto, Performance do Modelo no Cross-Validation, Modelo treinado
produto

{153066: [236142.25438296722, DecisionTreeRegressor()],
 420294: [12246.264994311254, DecisionTreeRegressor()],
 432552: [5350.382299376507, DecisionTreeRegressor()],
 177306: [14161.882421114058, DecisionTreeRegressor()],
 170478: [7875.00515154222, DecisionTreeRegressor()],
 125556: [14498.486724428489, DecisionTreeRegressor()],
 125424: [30354.70803472582, DecisionTreeRegressor()],
 153096: [118836.40393943973, DecisionTreeRegressor()],
 420282: [19974.71200531103, DecisionTreeRegressor()],
 166128: [27775.50251537527, DecisionTreeRegressor()],
 432588: [36602.032361299054, DecisionTreeRegressor()],
 432576: [15728.468655127183, DecisionTreeRegressor()],
 432582: [9412.857559446134, DecisionTreeRegressor()],
 153084: [152060.94152019627, DecisionTreeRegressor()],
 420288: [21839.107603067718, DecisionTreeRegressor()],
 170370: [13628.146333788567, DecisionTreeRegressor()],
 432528: [11148.249638353946, DecisionTreeRegressor()],
 420510: [34978.61184572204, DecisionTreeRegressor()],


In [9]:
# Previsão da variável target no conjunto de dados de teste, ciclos: 202016,202017,202101.
for i in tqdm(df_grupo.Grupo.index):
    
    df_aux = df_test[df_test['COD_MATERIAL'] == i]
    df_aux = df_aux.reset_index(drop=True)
    df_aux['ANO'] = df_aux['COD_CICLO'].apply(lambda x: int(str(x)[:4]))
    df_aux['CICLO'] = df_aux['COD_CICLO'].apply(lambda x: int(str(x)[4:]))
    df_aux['FLG_CAMPANHA_MKT_A'] = df_aux['FLG_CAMPANHA_MKT_A'].apply(lambda x: preprocessamento_indicadores(x))
    df_aux['FLG_CAMPANHA_MKT_B'] = df_aux['FLG_CAMPANHA_MKT_B'].apply(lambda x: preprocessamento_indicadores(x))
    df_aux['FLG_CAMPANHA_MKT_C'] = df_aux['FLG_CAMPANHA_MKT_C'].apply(lambda x: preprocessamento_indicadores(x))
    df_aux['FLG_CAMPANHA_MKT_D'] = df_aux['FLG_CAMPANHA_MKT_D'].apply(lambda x: preprocessamento_indicadores(x))
    df_aux['PCT_DESCONTO'].fillna(0,inplace=True)
    df_aux = df_aux.drop(['COD_CICLO','COD_MATERIAL','QT_VENDA'],axis=1)
    
    for j, k in zip(df_test[df_test['COD_MATERIAL'] == i].index, [0,1,2]):
        df_test.at[j,'QT_VENDA'] = produto[i][1].predict(df_aux.loc[k,:].values.reshape(1,-1))[0]
    

  0%|                                                                                          | 0/329 [00:00<?, ?it/s]C:\Users\renan\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\renan\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\renan\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\renan\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor 

C:\Users\renan\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\renan\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\renan\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\renan\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\renan\AppData\Local\Continuum\anaco

C:\Users\renan\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\renan\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\renan\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\renan\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\renan\AppData\Local\Continuum\anaco

C:\Users\renan\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\renan\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\renan\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\renan\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\renan\AppData\Local\Continuum\anaco

C:\Users\renan\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\renan\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\renan\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\renan\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\renan\AppData\Local\Continuum\anaco

...

C:\Users\renan\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\renan\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\renan\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\renan\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\renan\AppData\Local\Continuum\anaco

C:\Users\renan\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\renan\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\renan\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\renan\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
 90%|███████████████████████████████████████

 93%|██████████████████████████████████████████████████████████████████████████▍     | 306/329 [00:03<00:00, 94.25it/s]C:\Users\renan\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\renan\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\renan\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\renan\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor 

C:\Users\renan\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\renan\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\renan\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\renan\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\renan\AppData\Local\Continuum\anaco

C:\Users\renan\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  "X does not have valid feature names, but"
100%|████████████████████████████████████████████████████████████████████████████████| 329/329 [00:03<00:00, 96.27it/s]


In [10]:
df_test.to_excel('df_test_produtos_com_ciclos_completos.xlsx')